In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-06,France,133688.0,0.0,25872.0,0.0,0.0,0.0
0,2020-05-06,China,84052.0,0.0,4663.0,0.0,0.0,0.0
0,2020-05-06,Italy,214204.0,0.0,29503.0,0.0,0.0,0.0
0,2020-05-06,Spain,219764.0,0.0,25747.0,0.0,0.0,0.0
0,2020-05-06,United States,1228039.0,0.0,72618.0,0.0,0.0,0.0
0,2020-05-06,World,3705644.0,0.0,261219.0,0.0,0.0,0.0
0,2020-05-06,United Kingdom,199277.0,0.0,29809.0,0.0,0.0,0.0
0,2020-05-06,Germany,165397.0,0.0,7151.0,0.0,0.0,0.0
0,2020-05-06,Iran,101198.0,0.0,6420.0,0.0,0.0,0.0
0,2020-05-06,Turkey,130862.0,0.0,3573.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-07,France,133688.0,137150.0,25872.0,25809.0,-3462.0,63.0
0,2020-05-07,China,84052.0,83970.0,4663.0,4637.0,82.0,26.0
0,2020-05-07,Italy,214204.0,214457.0,29503.0,29684.0,-253.0,-181.0
0,2020-05-06,Spain,219764.0,220325.0,25747.0,25857.0,-561.0,-110.0
0,2020-05-07,United States,1228039.0,1228603.0,72618.0,73431.0,-564.0,-813.0
0,2020-05-07,World,3705644.0,3713796.0,261219.0,263288.0,-8152.0,-2069.0
0,2020-05-07,United Kingdom,199277.0,201201.0,29809.0,30076.0,-1924.0,-267.0
0,2020-05-07,Germany,165397.0,166091.0,7151.0,7119.0,-694.0,32.0
0,2020-05-07,Iran,101198.0,101650.0,6420.0,6418.0,-452.0,2.0
0,2020-05-07,Turkey,130862.0,131744.0,3573.0,3584.0,-882.0,-11.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")